In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
cfgList=[]
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData(basePath):
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()

path_projData='/project/s1174/lyan/code/projectData/'

basePath_NJN=path_projData+'nucleon_sigma_term/cA211.53.24/NJN/proj/'
basePath_NJNpi=path_projData+'nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/proj/'
basePath_loop=path_projData+'nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/data_post_merged/'

with h5py.File(basePath_NJN+'NJN.h5') as f:
    cfgList_NJN=list(yu.deepKey(f,3).keys())

with h5py.File(basePath_NJNpi+'N_2pt.h5') as f:
    cfgList_NJNpi=list(yu.deepKey(f,3).keys())

cfgList=list( set(cfgList_NJN) & set(cfgList_NJNpi) )
# cfgList=cfgList_NJNpi

cfgList.sort()
Ncfg=len(cfgList)
print(Ncfg)


filesDic={}
files2ptKey=[]
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['NJN']
filesDic['NJNpi']=['NJN-pi0i']
loadData(basePath_NJN)

filesDic={}
files2ptKey=['NN','NNpi','NpiN','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T','N_2pt-pi0i']
filesDic['NpiN']=['D1ii','N_2pt-pi0f']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt','N_2pt-pi0f-pi0i','T-pi0f','D1ii-pi0i']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_2pt-j']
filesDic['NJNpi']=['B','W','Z','N_2pt-j-pi0i','N_2pt-j&pi0i','T-j']
loadData(basePath_NJNpi)

# filesDic={}
# files2ptKey=['NN','NNpi','NpiN','NpiNpi']
# filesDic['NN']=['N_2pt']
# filesDic['NNpi']=['T']
# filesDic['NpiN']=['D1ii']
# filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
# files3ptKey=['NJNpi']
# filesDic['NJNpi']=['B','W','Z']
# loadData(basePath_NJNpi)


data['others']={}
with h5py.File(basePath_loop+'insertLoop.h5') as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
with h5py.File(basePath_loop+'pi0Loop.h5') as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])

In [ ]:
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

diags_key={}
diags_all=set(); diags_pi0Loop=set(); diags_jLoop=set()
for key in data.keys():
    if key in ['others']:
        continue
    diags_key[key] = set(data[key])
    diags_all = diags_all | diags_key[key]
    for diag in data[key]:
        t=diag.split('-')
        if 'pi0i' in t or 'pi0f' in t:
            diags_pi0Loop.add(diag)
        if 'j' in t:
            diags_jLoop.add(diag)
diags_loopful = diags_pi0Loop | diags_jLoop
diags_loopless = diags_all - diags_loopful
diags_all_2pt = diags_key['NN'] | diags_key['NNpi'] | diags_key['NpiNpi']
diags_all_3pt = diags_all - diags_all_2pt


Ntime=yu.deepKey(data['NN'],5).shape[1]

def getDat2pt(opa,opb,proj2,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['p'] and tpb in ['p']:
        key='NN'
        return data[key]['N_2pt'][proja][projb][proj2][tpa+'_'+tpb]+np.zeros([Ncfg,Ntime])
    elif tpa in ['p']:
        key='NNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['T']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0i']:
                return [tpa+'_p'] if tpb =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    elif tpb in ['p']:
        key='NpiN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['D1ii']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0f']:
                return ['p_'+tpb] if tpa =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    else:
        key='NpiNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B_2pt','W_2pt','Z_2pt']:
                return [tpa+'_'+tpb]
            if diag in ['M_correct_2pt']:
                return [tpa+'_'+tpa] if tpa==tpb else []
            if diag in ['N_2pt-pi0f-pi0i']:
                return ['p_p'] if tpa==tpb=='p&pi0' else []
            if diag in ['T-pi0f']:
                return ['p_'+tpb] if tpa=='p&pi0' else []
            if diag in ['D1ii-pi0i']:
                return [tpa+'_p'] if tpb=='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NpiNpi*np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])

def getDat2ptMat(ops,proj2,diags):
    t=np.transpose([[getDat2pt(opa,opb,proj2,diags) for opb in ops] for opa in ops],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(opa,opb,Gc,iso,proj2,tf,diags):
    (tpa,proja)=opa; (tpb,projb)=opb
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    if tpa in ['p'] and tpb in ['p']:
        key='NJN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['NJN','N_2pt-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont) for diag in t_diags for cont in get_conts(diag)]
        return np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])       
    elif tpa in ['p']:
        key='NJNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B','W','Z']:
                return [tpa+'_j'+iso+'_'+tpb+'_deltat_'+str(tf)]
            if diag in ['N_2pt-j&pi0i','N_2pt-j-pi0i','NJN-pi0i']:
                return [tpa+'_p'+'_j'+iso+'_deltat_'+str(tf)] if tpb == 'p&pi0' else []
            if diag in ['T-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NJNpi*np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])
    elif tpb in ['p']:
        if projb[0] != (0,0,0):
            return np.zeros([Ncfg,tf-1])
        return np.conj(getDat3pt(opb,opa,Gc,iso,proj2,tf,diags)[:,::-1])
    else:
        return np.zeros([Ncfg,tf-1])
    
def getDat3ptMat(opas,opbs,Gc,iso,proj2,tf,diags):
    t=np.transpose([[getDat3pt(opa,opb,Gc,iso,proj2,tf,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(opas,opbs,Gc,iso,proj2,diags):
    pta=opas[0][1][0]; ptb=opbs[0][1][0]
    assert(pta==(0,0,0)); pc=ptb
    if pc==(0,0,0) and proj2=='Pzt':
        proj2='Pz'
    dat2ptA=getDat2ptMat(opas,diags)
    dat2ptB=getDat2ptMat(opbs,diags)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(opas,opbs,Gc,iso,proj2,tf,diags)
    if (Gc,iso) == ('id','+'):
        dat_jLoop=data['others']['u+d_id_vev']
    elif (Gc,iso) == ('g5','-'):
        dat_jLoop=data['others']['u-d_g5_vev']
    else:
        dat_jLoop=np.zeros(Ncfg)

    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[np.array([dat3pt[tf][cfg] for tf in tfList],dtype=object)]+[dat_jLoop[cfg]],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    return t

# # test
# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# proj2=(pt1,('N1\\pi1','a')); proj22=(pt2,('N1\\pi0','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',proj2); op22=('n&pi+',proj22)
# op3=('p&pi0',proj2); op32=('p&pi0',proj22)

# getDat3pt(op1,op32,'id','+','Pzt',10,diags_all).shape
# t=getDat([op1,op2,op3],[op12,op22,op32],'id','+','Pzt',10,diags_all)

In [ ]:
def run2pt(opa,opb,diags,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False):
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2)
        if title == None:
            title=opa[0]+'_'+opb[0]
        else:
            title=opa[0]+'_'+opb[0]+': '+title
        fig.suptitle(title)

    for proj2 in ['P01','P02','P0']:
        colorDic={
            'P01':'r','P02':'b','P0':'g'
        }
        dat=preFactor*getDat2pt(opa,opb,proj2,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,proj2,diags)+np.conj(getDat2pt(opb,opa,proj2,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color=colorDic[proj2],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].grid()
        axs[irow,icol].set_xticks(range(0,tmax,2))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color=colorDic[proj2],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].grid()
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].set_ylim(ylim2)
        if opa[1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif opa[1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange')

In [ ]:
def run2ptGEVP(ops,diags,pj2,title=None,ylim1=[.4,1.1],ylim2=[1e-4,1.1],axs=None):
    N_op=len(ops); tRef=4
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2)
        if title == None:
            title=''
        else:
            title=title
        fig.suptitle(title)

    colorDic={'P01':'r','P02':'b','P0':'g'}
    yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
    fmtDic={0:'s',1:'d',2:'o'}

    for proj2 in [pj2]:
        dat=getDat2ptMat(ops,proj2,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].grid()
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].set_ylim(ylim1)

        if ops[0][1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif ops[0][1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 

        icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            t=eVecs[:,:,0].T
            return np.abs(t)
            return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        mean0=mean[0,tmin:tmax]

        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0

            # temp_min=8
            # t_factor=1 if i!=1 else 1/np.sqrt(2)
            # print(i)
            # print(yu.npRound(tMean[temp_min:]*t_factor,3))
            # print(yu.npRound(tErr[temp_min:]*t_factor,3))

            axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt=fmtDic[i],color='r',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].grid()
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].set_ylim(ylim2)
        axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

# 2pt

In [ ]:
pt=(0,0,0)
proj1=(pt,'a'); proj2=(pt,('N1\\pi1','a'))

opa=('p',proj1); opb=('n&pi+',proj2)
diags=diags_all
run2pt(opa,opb,diags,'imag',preFactor=-1j)
run2pt(opb,opa,diags,'imag',preFactor=-1j)

opa=('p',proj1); opb=('p&pi0',proj2)
diags=diags_all
run2pt(opa,opb,diags,'imag',preFactor=-1j)
run2pt(opb,opa,diags,'imag',preFactor=-1j)

In [ ]:
pt=(0,0,0)
proj=(pt,('N1\\pi1','a'))
opa=('n&pi+',proj); opb=('p&pi0',proj)

print(diags_key['NpiNpi']) 
diags=diags_all
run2pt(opa,opb,diags) 
run2pt(opb,opa,diags)

# 2ptGEVP

In [ ]:
pt=(0,0,1)
proj1=(pt,'a'); proj2=(pt,('N1\\pi0','a'))
opa=('p',proj1); opb=('p&pi0',proj2)

diags=diags_all

run2pt(opa,opb,diags,'real',ylim1='sym')
run2pt(opa,opb,diags,'imag',preFactor=-1j)

ops=[opa,opb]
run2ptGEVP(ops,diags,'P0','P0')
run2ptGEVP(ops,diags,'P01','l1')
run2ptGEVP(ops,diags,'P02','l2')

In [ ]:
pt=(0,0,1)
proj1=(pt,'a'); proj2=(pt,('N1\\pi0','a'))
opa=('p',proj1); opb=('n&pi+',proj2)

diags=diags_all

run2pt(opa,opb,diags,'real',ylim1='sym')
run2pt(opa,opb,diags,'imag',preFactor=-1j)

ops=[opa,opb]
run2ptGEVP(ops,diags,'P0','P0')
run2ptGEVP(ops,diags,'P01','l1')
run2ptGEVP(ops,diags,'P02','l2')

In [ ]:
pt=(0,0,0)
proj1=(pt,'a'); proj2=(pt,('N1\\pi1','a'))
opa=('p',proj1); opb=('n&pi+',proj2); opc=('p&pi0',proj2)

diags=diags_all

# run2pt(opa,opb,diags,'real',ylim1='sym')
# run2pt(opa,opb,diags,'imag',preFactor=-1j)

ops=[opa,opb]
run2ptGEVP(ops,diags,'P0','n&pi+')
ops=[opa,opc]
run2ptGEVP(ops,diags,'P0','p&pi0')
ops=[opa,opb,opc]
run2ptGEVP(ops,diags,'P0','both')